Importamos librerias

In [1]:
import pyRofex as pyRofex
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as datetime

# Import Data

## Log in

Hacemos el Login para poder traer datos de la API de Rofex

In [2]:
pyRofex.initialize(user="luciomassimi3968",
                   password="feulrX6%",
                   account="REM3968",
                   environment=pyRofex.Environment.REMARKET)

## Spot

Seteamos los parametros para traer datos del spot.

In [3]:
tickers_spot = ['GGAL.BA','PAMP.BA','YPFD.BA','ARS=X']

start = datetime.datetime(2022,1,1) # Starting date in format (year,month,day)
end = datetime.datetime.today() # Ending date

In [4]:
# spot_df = pd.DataFrame()
# for ticker in tickers_spot:
#     """
#     This will make a dataframe with the "Close" of each of our stocks
#     """
#     spot_df[ticker] = yf.Ticker(ticker).history(start=end,end=end)['Close']

# spot_df.columns =['GGAL','PAMP','YPFD','DLR']
# spot_df.tail()

In [5]:
spot_df = pd.DataFrame()
for ticker in tickers_spot:
    """
    This will make a dataframe with the "Close" of each of our stocks
    """
    spot_df[ticker] = yf.Ticker(ticker).history(start=start,end=end)['Close']

spot_df.columns =['GGAL','PAMP','YPFD','DLR']
spot_df.tail()

,GGAL,PAMP,YPFD,DLR
Date,,,,
2022-01-19,191.050003,160.149994,866.299988,104.122375
2022-01-20,195.699997,164.850006,877.349976,104.201080
2022-01-21,189.399994,166.250000,851.650024,104.261841
2022-01-24,181.550003,163.800003,826.200012,104.350235
2022-01-25,196.000000,173.500000,886.750000,104.610001


In [6]:
# spot.T.iloc[:,-1]

In [7]:
spot_last= spot_df.iloc[-1]
spot_last

GGAL    196.000000
PAMP    173.500000
YPFD    886.750000
DLR     104.610001
Name: 2022-01-25 00:00:00, dtype: float64

## Futures

### Import GGAL, PAMP, YFD, DLR/FUT

Seteamos los tickers de los activos que queremos traer, en este caso traemos los futuros de GGAL, PAMP, YFD, y todos los futuros DLR/FUT de 2022

In [8]:
futuros_dolar=['DLR/ENE22','DLR/FEB22','DLR/MAR22','DLR/ABR22','DLR/MAY22','DLR/JUN22','DLR/JUL22','DLR/AGO22','DLR/SEP22','DLR/OCT22','DLR/NOV22']#,'DLR/DIC22']
futuro_ggal=['GGAL/FEB22']
futuro_pamp=['PAMP/FEB22']
futuro_ypfd=['YPFD/FEB22']

tickers= futuros_dolar  + futuro_pamp + futuro_ypfd + futuro_ggal
depth=5

Usamos la funcion 'get_market_data' para traer la data de la rest API de rofex, y la almacenamos en un diccionario para luego poder extraer los datos que necesitemos

In [9]:
resp_dict = {}

for instrument in tickers:
    resp_dict[f"{instrument}"] = pyRofex.get_market_data(ticker = instrument, depth = depth)
    
resp_dict

{'DLR/ENE22': {'status': 'OK',
  'marketData': {'OI': {'price': None, 'size': 103485, 'date': 1630195200000},
   'NV': 20019000,
   'IV': None,
   'OP': 105.2,
   'LO': 105.1,
   'LA': {'price': 106.41, 'size': 999, 'date': 1643134368403},
   'HI': 106.59,
   'SE': {'price': 116.0, 'size': None, 'date': 1630281600000},
   'EV': 2116488710.0,
   'BI': [{'price': 105.14, 'size': 3534},
    {'price': 104.99, 'size': 2534},
    {'price': 104.74, 'size': 3534},
    {'price': 104.42, 'size': 2184},
    {'price': 103.98, 'size': 3534}],
   'CL': {'price': 105.2, 'size': None, 'date': 1642982400000},
   'TV': 20019.0,
   'OF': []},
  'depth': 5,
  'aggregated': True},
 'DLR/FEB22': {'status': 'OK',
  'marketData': {'OI': {'price': None, 'size': 74681, 'date': 1630195200000},
   'NV': 63000,
   'IV': None,
   'OP': 108.2,
   'LO': 108.2,
   'LA': {'price': 108.68, 'size': 11, 'date': 1643133487719},
   'HI': 108.73,
   'SE': {'price': 120.4, 'size': None, 'date': 1630281600000},
   'EV': 684544

### BID - OFFER per instrument

Del diccionario de datos que creamos a partir del query, extraemos los precios del BID y OFFER de los distintos activos. A veces como la liquidez es muy limitada, no hay puntas en el BID ni en el OFFER. Por esto tambien agregue un nested "try - Except" por si es totalmente necesario traer algun dato. Esto a la hora de tradear o correr el bot no deberia hacerse, y solo se deberian tomar los datos de BID y OFFER. En este caso lo hago para que se vean mas datos cuando se corre la notebook.

In [10]:
# #BID
# bid={}
# for instrument in tickers:
#     try:
#         bid[instrument] = resp_dict[instrument]['marketData']['BI'][0]['price']
#     except:
#         print(f"{instrument} has no BID")
#         bid[instrument] = np.nan

# bid

In [11]:
#BID
bid={}
for instrument in tickers:
    #Trato de traer los valores del precio
    try:
        bid[instrument] = resp_dict[instrument]['marketData']['BI'][0]['price']
    
    #si no trae valores de precio y necesito probar algo de las funciones que siguen, puedo tratar de importar los
    #datos del cierre o del ultimo precio
    except:
#         try:
#             print(f"{instrument} has no BID")
#             bid[instrument] = resp_dict[instrument]['marketData']['LA'][0]['price']
            
#         except:
#             try:
#                 bid[instrument] = resp_dict[instrument]['marketData']['CL']['price']
        
#             except:
                print(f"{instrument} has no BID")
                bid[instrument] = np.nan

bid

GGAL/FEB22 has no BID


{'DLR/ENE22': 105.14,
 'DLR/FEB22': 108.68,
 'DLR/MAR22': 114.17,
 'DLR/ABR22': 118.85,
 'DLR/MAY22': 124.0,
 'DLR/JUN22': 128.4,
 'DLR/JUL22': 133.3,
 'DLR/AGO22': 138.6,
 'DLR/SEP22': 143.0,
 'DLR/OCT22': 147.7,
 'DLR/NOV22': 152.5,
 'PAMP/FEB22': 175.45,
 'YPFD/FEB22': 905.0,
 'GGAL/FEB22': nan}

In [12]:
# #OFFER
# offer={}
# for instrument in tickers:
#     try:
#         offer[instrument] = resp_dict[instrument]['marketData']['OF'][0]['price']
#     except:
#         print(f"{instrument} has no OFFER")
#         offer[instrument] = np.nan

# offer

In [13]:
#OFFER
offer={}
for instrument in tickers:
    #Trato de traer los valores del precio
    
    try:
        offer[instrument] = resp_dict[instrument]['marketData']['OF'][0]['price']
        
    except:
        
        #si no trae valores de precio y necesito probar algo de las funciones que siguen, puedo tratar de importar los
        #datos del cierre o del ultimo precio
#         try:
#             print(f"{instrument} has no OFFER")
#             offer[instrument] = resp_dict[instrument]['marketData']['LA'][0]['price']
            
#         except:
#             try:
#                 offer[instrument] = resp_dict[instrument]['marketData']['CL']['price']
        
#             except:
                print(f"{instrument} has no OFFER")
                offer[instrument] = np.nan

offer

DLR/ENE22 has no OFFER
GGAL/FEB22 has no OFFER


{'DLR/ENE22': nan,
 'DLR/FEB22': 108.71,
 'DLR/MAR22': 114.71,
 'DLR/ABR22': 118.89,
 'DLR/MAY22': 124.05,
 'DLR/JUN22': 128.98,
 'DLR/JUL22': 133.7,
 'DLR/AGO22': 139.09,
 'DLR/SEP22': 144.0,
 'DLR/OCT22': 148.77,
 'DLR/NOV22': 153.0,
 'PAMP/FEB22': 176.0,
 'YPFD/FEB22': 905.25,
 'GGAL/FEB22': nan}

Creo el dataframe con los valores de BID y OFFER para cada uno de los activos que pido

In [14]:
fut = pd.DataFrame({"bid":bid,"offer":offer})
fut

,bid,offer
DLR/ENE22,105.14,NaN
DLR/FEB22,108.68,108.71
DLR/MAR22,114.17,114.71
DLR/ABR22,118.85,118.89
DLR/MAY22,124.00,124.05
DLR/JUN22,128.40,128.98
DLR/JUL22,133.30,133.70
DLR/AGO22,138.60,139.09
DLR/SEP22,143.00,144.00
DLR/OCT22,147.70,148.77


Si es necesario, se puede agregar la comision de compra y venta para el futuro para saber cuales serian las tasas reales en caso de operar
(lo mismo se deberia hacer para el spot)

In [15]:
#comision
# comision = .005

# fut['offer']=fut['offer']*(1+comision)
# fut['bid']=fut['bid']*(1-comision)
# fut

# Modify Data

## Defining functions: 

### Tasas

Creamos distintas funciones para obtener la tasa. La tasa va a variar dependiendo si usamos tasa de compounding continuo(que es lo que se suele usar en derivados), si se calcula TNA, TEA, etc. Para esto se crea una funcion que engloba a todas las otras funciones, y cambiando la variable "TIPO" vamos a poder elegir entre una u otra. Tambien se puede seleccionar el input temporal como dia o mes de ser necesario

In [16]:
spot_v=100
future_v=105
dias=30
mes=1

Tasa de compounding continuo, gralmente usada para derivados.

In [17]:
def tasa_continua_mes(spot,future,t=1):
    tasa=np.log(future/spot) / (t/12)
    return(tasa)

tasa_continua_mes(spot_v,future_v,mes)

0.5854819700331846

In [18]:
def tasa_continua_dias(spot,future,t=30):
    tasa=np.log(future/spot) / (t/365)
    return(tasa)

tasa_continua_dias(spot_v,future_v,dias)

0.5936136640614232

Tasa Nocional Anual, el equivalente seria la tasa que se ve en los plazo fijos

In [19]:
def tasa_tna_mes(spot,future,t=1):
    tasa=((future/spot)-1)*(12/t)
    return(tasa)

tasa_tna_mes(spot_v,future_v,mes)

0.6000000000000005

In [20]:
def tasa_tna_dias(spot,future,t=30):
    tasa=((future/spot)-1)*(365/t)
    return(tasa)

tasa_tna_dias(spot_v,future_v,dias)

0.6083333333333338

Tasa Efectiva Anual, es la tasa 'REAL' que estariamos percibiendo en el ano. La diferencia con la nocional, es que la nocional no toma en cuenta el compounding y en esta si se tiene en cuenta. 
Por ej, si ponemos un plazo fijo con TNA 30% a un ano, 100>130. Pero si en vez de ponerla a un ano se pone a la misma tasa TNA 30% pero cada mes, al final del ano se va a tener >130 ya que cada mes se va a tener un poco de retorno y ese retorno tambien se va a colocar. Por eso la TEA > TNA

In [21]:
def tasa_tea_mes(spot,future,t=30):
    tasa=((future/spot)**(12/t))-1
    return(tasa)

tasa_tea_mes(spot_v,future_v,mes)

0.7958563260221301

In [22]:
def tasa_tea_dias(spot,future,t=30):
    tasa=((future/spot)**(365/t))-1
    return(tasa)

tasa_tea_dias(spot_v,future_v,dias)

0.8105192164554333

La tasa directa es el % real sin anualizar

In [23]:
def tasa_directa(spot,future):
    tasa=(future/spot)-1
    return(tasa)

tasa_directa(spot_v,future_v)

0.050000000000000044

In [24]:
spot_v, future_v

(100, 105)

Esta funcion toma todas las otras funciones calculadoras de tasa y las junta en una sola para que sea mas simple seleccionar la tasa que se quiere usar. En vez de cambiar todas las funciones, se puede poner una variable como 'tipo' antes de correr las funciones y seleccionar el tipo de tasa que se quiere usar de esa manera. Cumpliria la funcion de 'switch'

In [25]:
# def tipo_tasa(tipo,period,spot,future,t=1):
    
#     if tipo.lower()== "continua":
        
#         if period.lower()=="mes":
#             return(tasa_continua_mes(spot,future,t))
        
#         elif period.lower()=="dias":
#             return(tasa_continua_dias(spot,future,t))
        
#         else:
#             print("Options: mes, dias")
            
            
#     elif tipo.lower()== "tna":
        
#         if period.lower()=="mes":
#             return(tasa_tna_mes(spot,future,t))
        
#         elif period.lower()=="dias":
#             return(tasa_tna_dias(spot,future,t))
        
#         else:
#             print("Options: mes, dias")
        
        
#     elif tipo.lower()== "tea":
        
#         if period.lower()=="mes":
#             return(tasa_tea_mes(spot,future,t))
        
#         elif period.lower()=="dias":
#             return(tasa_tea_dias(spot,future,t))
        
#         else:
#             print("Options: mes, dias")

In [26]:
def tipo_tasa(spot,future,t=30,tipo="continua",period="dias"):
    
    if tipo.lower()== "continua":
        
        if period.lower()=="mes":
            return(tasa_continua_mes(spot,future,t))
        
        elif period.lower()=="dias":
            return(tasa_continua_dias(spot,future,t))
        
        else:
            print("Options: mes, dias")
            
            
    elif tipo.lower()== "tna":
        
        if period.lower()=="mes":
            return(tasa_tna_mes(spot,future,t))
        
        elif period.lower()=="dias":
            return(tasa_tna_dias(spot,future,t))
        
        else:
            print("Options: mes, dias")
        
        
    elif tipo.lower()== "tea":
        
        if period.lower()=="mes":
            return(tasa_tea_mes(spot,future,t))
        
        elif period.lower()=="dias":
            return(tasa_tea_dias(spot,future,t))
        
        else:
            print("Options: mes, dias")
            
    elif tipo.lower()== 'directa':
        return(tasa_directa(spot,future))
    
    else:
        return("Error")

In [27]:
spot_v=104.261841
future_v=105.28
dias=10
mes=(10/30)

tipo_tasa(spot_v,future_v)

0.11823603914081394

In [28]:
spot_v=104.261841
future_v=105.28
dias=10
mes=(10/30)

tipo_tasa(spot=spot_v,
          future=future_v,
          t=dias,
          tipo='directa',
          period='dias')

0.00976540400816428

### Days until settlement

Es importante tener en cuenta que al calcular las tasas de manera diaria se tienen que usar los dias exactos desde que se hace la transaccion hasta el dia del settlement. Generalmente el settlement se da el ultimo dia habil del mes de vencimiento de futuro, pero esto no siempre va a coincidir con el ultimo dia real del mes. Es por esto que tener una funcion que calcula los dias exactos hasta el ultimo dia habil del mes o settlement, es muy importante porque con este input se puede calcular la tasa correcta.

In [29]:
import calendar
from datetime import date

#Calculo el ultimo dia habil del mes, teniendo ano y mes como input.
#De la ultima semana del mes, toma el maximo dia de entre 0:5(lun-vier)
def last_business_day_in_month(year: int, month: int):
    return(date(year,month,max(calendar.monthcalendar(year, month)[-1:][0][:5])))

#Calculo el ultimo dia habil del mes, tomamos la ecuacion anterior
def days_until_last_business_day(year:int,month:int):
    
    delta = last_business_day_in_month(year,month) - date.today()
    return(delta.days)   

days_until_last_business_day(2022,2)

34

###  Bid - Offer Rates

Tomo las funciones creadas anteriormente para calcular las tasas tomadoras y colocadoras usando los datos del spot y futuros importados

In [30]:
#Final DataFrame
tasas_anuales_bid_offer =pd.DataFrame(columns=['bid','offer'])


#List variables
month_list= ['ene','feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']
activo_tipo_list = ['DLR', 'GGAL','PAMP','YPFD']
activo_ano_list = ['22','23']
ano_list=[2022,2023]


# Function variables> elijo el tipo de tasa que se va a calcular
tipo='continua'
period='dias'


## Itero con las columnas'bid' y 'offer' del dataframe final
for column in tasas_anuales_bid_offer:
    ind=0
#     print("\n",column,"\n")
    
    ## Itero y tomo cada dato de 'bid' y 'offer', se usan los datos de los precios de los futuros
    for data in fut[column]:
#         print(f"data {data}")
        
        ## Itero para saber a que ano pertenece el futuro
        for ano_activo in activo_ano_list:
            
            ## Corroboro si el ano esta en el indice del precio a traer, y seteo el ano correspondiente
            ## para usar en la formula que calcula los dias hasta el settlement
            if ano_activo in fut[column].index[ind]:
                year=ano_list[activo_ano_list.index(ano_activo)]

#                 print(fut[column].index[ind])
                
                ## Corroboro el tipo de activo de donde viene la data, DLR,GGAL,PAMP o YPFD
                for tipo_activo in activo_tipo_list:

                    if tipo_activo in fut[column].index[ind]:
                        
                        ## Establezco el mes correspondiente a la data
                        for mes in month_list:


                            if mes in fut[column].index[ind].lower():
#                                 print(f"Mes: {mes}")

                                ## Teniendo el el precio del futuro,el ano y mes de settlement, y el tipo de activo 
                                ## del que quiero calcular la tasa, puedo traer el valor del spot, calcular los dias
                                ## hasta el settlement y calcular la tasa
            
                                #dataframe
                                index= fut[column].index[ind]
                                column= column

                                #function
                                spot= spot_df.iloc[-1][tipo_activo]
                                future= data
                                month = month_list.index(mes)+1


                                #output
                                tasas_anuales_bid_offer.loc[index,column] = round(tipo_tasa(spot=spot,
                                                                                      future=future,
                                                                                      t=days_until_last_business_day(year,month),
                                                                                      tipo=tipo,
                                                                                      period=period),6)
#                                 print(f"Tipo Activo: {tipo_activo}")
#                                 print(f"Spot: {spot}")
#                                 print(f"Future: {future}")
#                                 print(f"Year: {year}")
#                                 print(f"Month count: {month}")
#                                 print(f"Dias faltan: {days_until_last_business_day(year,month)}")
#                                 print(f"Tipo: {tipo}")
#                                 print(f"Periodo: {period}")
#                                 print(f"Tasa: {tasas_anuales_bid_offer.loc[index,column]}\n")

#             else:
#                 print(f"Other Year {ano_activo}")
                
#         print(f"Ind: {ind}\n")
        ind+=1

            
# tasas_anuales_bid_offer

Tasa colocadora: bid, Tasa tomadora: offer

In [31]:
# tasas_anuales_bid_offer

In [32]:
# arbitraje_dict={}
# count=0
# for offer in tasas_anuales_bid_offer['offer']:
#     for bid in tasas_anuales_bid_offer['bid']:
#         if bid>offer:
#             print(f"Bid: {tasas_anuales_bid_offer[tasas_anuales_bid_offer['bid'] == bid].index[0]} {bid},\nOffer: {tasas_anuales_bid_offer[tasas_anuales_bid_offer['offer'] == offer].index[0]} {offer}\n")
#             arbitraje_dict.update({f"{tasas_anuales_bid_offer[tasas_anuales_bid_offer['bid'] == bid].index[0]}-{count}": (bid, (tasas_anuales_bid_offer[tasas_anuales_bid_offer['offer'] == offer].index[0], offer))})
#             count+=1

In [33]:
# arbitraje_dict

In [34]:
# arbitraje_df = pd.DataFrame(arbitraje_dict).T.sort_index()
# arbitraje_df.columns = ['bid','offer']
# arbitraje_df

# Websocket


In [35]:
instrument = tickers

In [36]:
prices = pd.DataFrame(columns=["activo", "bid", "offer", "last"])
prices.set_index('activo', inplace=True)

In [37]:
def tasas_futuros(df):
    
    #Final DataFrame
    tasas_anuales_bid_offer =pd.DataFrame(columns=['bid','offer'])


    #List variables
    month_list= ['ene','feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']
    activo_tipo_list = ['DLR', 'GGAL','PAMP','YPFD']
    activo_ano_list = ['22','23']
    ano_list=[2022,2023]


    # Function variables> elijo el tipo de tasa que se va a calcular
    tipo='continua'
    period='dias'


    ## Itero con las columnas'bid' y 'offer' del dataframe final
    for column in tasas_anuales_bid_offer:
        ind=0
    #     print("\n",column,"\n")

        ## Itero y tomo cada dato de 'bid' y 'offer', se usan los datos de los precios de los futuros
        for data in df[column]:
    #         print(f"data {data}")

            ## Itero para saber a que ano pertenece el futuro
            for ano_activo in activo_ano_list:

                ## Corroboro si el ano esta en el indice del precio a traer, y seteo el ano correspondiente
                ## para usar en la formula que calcula los dias hasta el settlement
                if ano_activo in df[column].index[ind]:
                    year=ano_list[activo_ano_list.index(ano_activo)]

    #                 print(df[column].index[ind])

                    ## Corroboro el tipo de activo de donde viene la data, DLR,GGAL,PAMP o YPFD
                    for tipo_activo in activo_tipo_list:

                        if tipo_activo in df[column].index[ind]:

                            ## Establezco el mes correspondiente a la data
                            for mes in month_list:


                                if mes in df[column].index[ind].lower():
    #                                 print(f"Mes: {mes}")

                                    ## Teniendo el el precio del futuro,el ano y mes de settlement, y el tipo de activo 
                                    ## del que quiero calcular la tasa, puedo traer el valor del spot, calcular los dias
                                    ## hasta el settlement y calcular la tasa

                                    #dataframe
                                    index= df[column].index[ind]
                                    column= column

                                    #function
                                    spot= spot_df.iloc[-1][tipo_activo]
                                    future= data
                                    month = month_list.index(mes)+1


                                    #output
                                    tasas_anuales_bid_offer.loc[index,column] = round(tipo_tasa(spot=spot,
                                                                                          future=future,
                                                                                          t=days_until_last_business_day(year,month),
                                                                                          tipo=tipo,
                                                                                          period=period),6)
    #                                 print(f"Tipo Activo: {tipo_activo}")
    #                                 print(f"Spot: {spot}")
    #                                 print(f"Future: {future}")
    #                                 print(f"Year: {year}")
    #                                 print(f"Month count: {month}")
    #                                 print(f"Dias faltan: {days_until_last_business_day(year,month)}")
    #                                 print(f"Tipo: {tipo}")
    #                                 print(f"Periodo: {period}")
    #                                 print(f"Tasa: {tasas_anuales_bid_offer.loc[index,column]}\n")

    #             else:
    #                 print(f"Other Year {ano_activo}")

    #         print(f"Ind: {ind}\n")
            ind+=1


    return(tasas_anuales_bid_offer)

In [38]:
def market_data_handler(message):
    global prices
    print("Market Data Message Received: {0}".format(message))
    last = None if not message["marketData"]["LA"] else message["marketData"]["LA"]["price"]
    prices.loc[message["instrumentId"]['symbol']] = [
        message["marketData"]["BI"][0]["price"],
        message["marketData"]["OF"][0]["price"],
        last
    ]
    display(prices)
    display(tasas_futuros(prices))
    display(arbitraje_tasas(tasas_futuros(prices)))

In [39]:
def arbitraje_tasas(df):
    arbitraje_dict={}
    count=0
    for offer in df['offer']:
        for bid in df['bid']:
            if bid>offer:
#                 print(f"Bid: {df[df['bid'] == bid].index[0]} {bid},\nOffer: {df[df['offer'] == offer].index[0]} {offer}\n")
                arbitraje_dict.update({f"{df[df['bid'] == bid].index[0]}-{count}": (bid, (df[df['offer'] == offer].index[0], offer))})
                count+=1
    arbitraje_df = pd.DataFrame(arbitraje_dict).T.sort_index()
    arbitraje_df.columns = ['bid','offer']
    arbitraje_df
    return(arbitraje_df)

In [40]:
pyRofex.init_websocket_connection(market_data_handler=market_data_handler)

In [41]:
# Subscribes to receive market data messages
pyRofex.market_data_subscription(
    tickers=instrument,
    entries=[
        pyRofex.MarketDataEntry.BIDS,
        pyRofex.MarketDataEntry.OFFERS,
        pyRofex.MarketDataEntry.LAST]
)

In [42]:
# pyRofex.close_websocket_connection()

Market Data Message Received: {'type': 'Md', 'timestamp': 1643140232478, 'instrumentId': {'marketId': 'ROFX', 'symbol': 'DLR/AGO22'}, 'marketData': {'BI': [{'price': 138.6, 'size': 20}], 'OF': [{'price': 139.09, 'size': 10}], 'LA': {'price': 138.96, 'size': 10, 'date': 1643133611896}}}


,bid,offer,last
activo,,,
DLR/AGO22,138.6,139.09,138.96


,bid,offer
DLR/AGO22,0.471073,0.476981


Market Data Message Received: {'type': 'Md', 'timestamp': 1643140232480, 'instrumentId': {'marketId': 'ROFX', 'symbol': 'DLR/ABR22'}, 'marketData': {'BI': [{'price': 118.85, 'size': 565}], 'OF': [{'price': 118.89, 'size': 50}], 'LA': {'price': 118.87, 'size': 3, 'date': 1643130704677}}}


,bid,offer,last
activo,,,
DLR/AGO22,138.60,139.09,138.96
DLR/ABR22,118.85,118.89,118.87


...

,bid,offer,last
activo,,,
DLR/AGO22,138.60,139.09,138.96
DLR/ABR22,118.85,118.89,118.87
DLR/FEB22,108.68,108.71,108.68


,bid,offer
DLR/AGO22,0.471073,0.476981
DLR/ABR22,0.495558,0.496864
DLR/FEB22,0.409751,0.412714


,bid,offer
DLR/ABR22-0,0.495558,"(DLR/AGO22, 0.476981)"
DLR/ABR22-2,0.495558,"(DLR/FEB22, 0.412714)"
DLR/AGO22-1,0.471073,"(DLR/FEB22, 0.412714)"


Market Data Message Received: {'type': 'Md', 'timestamp': 1643140232481, 'instrumentId': {'marketId': 'ROFX', 'symbol': 'DLR/JUN22'}, 'marketData': {'BI': [{'price': 128.4, 'size': 2}], 'OF': [{'price': 128.98, 'size': 250}], 'LA': {'price': 128.99, 'size': 3, 'date': 1643133214722}}}


,bid,offer,last
activo,,,
DLR/AGO22,138.60,139.09,138.96
DLR/ABR22,118.85,118.89,118.87
DLR/FEB22,108.68,108.71,108.68
DLR/JUN22,128.40,128.98,128.99


In [43]:
# arbitraje_tasas(tasas_futuros(prices))

,bid,offer
DLR/AGO22,0.471073,0.476981
DLR/ABR22,0.495558,0.496864
DLR/FEB22,0.409751,0.412714
DLR/JUN22,0.47944,0.489985


,bid,offer
DLR/ABR22-0,0.495558,"(DLR/AGO22, 0.476981)"
DLR/ABR22-3,0.495558,"(DLR/FEB22, 0.412714)"
DLR/ABR22-5,0.495558,"(DLR/JUN22, 0.489985)"
DLR/AGO22-2,0.471073,"(DLR/FEB22, 0.412714)"
DLR/JUN22-1,0.47944,"(DLR/AGO22, 0.476981)"
DLR/JUN22-4,0.47944,"(DLR/FEB22, 0.412714)"


Market Data Message Received: {'type': 'Md', 'timestamp': 1643140232481, 'instrumentId': {'marketId': 'ROFX', 'symbol': 'DLR/MAY22'}, 'marketData': {'BI': [{'price': 124.0, 'size': 100}], 'OF': [{'price': 124.05, 'size': 70}], 'LA': {'price': 123.95, 'size': 6, 'date': 1643133288617}}}


,bid,offer,last
activo,,,
DLR/AGO22,138.60,139.09,138.96
DLR/ABR22,118.85,118.89,118.87
DLR/FEB22,108.68,108.71,108.68
DLR/JUN22,128.40,128.98,128.99
DLR/MAY22,124.00,124.05,123.95


,bid,offer
DLR/AGO22,0.471073,0.476981
DLR/ABR22,0.495558,0.496864
DLR/FEB22,0.409751,0.412714
DLR/JUN22,0.47944,0.489985
DLR/MAY22,0.492583,0.493751


...

,bid,offer
DLR/ABR22-0,0.495558,"(DLR/AGO22, 0.476981)"
DLR/ABR22-14,0.495558,"(DLR/JUN22, 0.489985)"
DLR/ABR22-17,0.495558,"(DLR/MAY22, 0.493751)"
DLR/ABR22-19,0.495558,"(YPFD/FEB22, 0.221663)"
DLR/ABR22-29,0.495558,"(DLR/NOV22, 0.449102)"
DLR/ABR22-37,0.495558,"(DLR/OCT22, 0.460714)"
DLR/ABR22-42,0.495558,"(DLR/JUL22, 0.484087)"
DLR/ABR22-46,0.495558,"(DLR/SEP22, 0.470341)"
DLR/ABR22-6,0.495558,"(DLR/FEB22, 0.412714)"
DLR/AGO22-18,0.471073,"(YPFD/FEB22, 0.221663)"


Market Data Message Received: {'type': 'Md', 'timestamp': 1643140232483, 'instrumentId': {'marketId': 'ROFX', 'symbol': 'PAMP/FEB22'}, 'marketData': {'BI': [{'price': 175.45, 'size': 5}], 'OF': [{'price': 176.0, 'size': 5}], 'LA': {'price': 175.5, 'size': 3, 'date': 1643138839611}}}


,bid,offer,last
activo,,,
DLR/AGO22,138.60,139.09,138.96
DLR/ABR22,118.85,118.89,118.87
DLR/FEB22,108.68,108.71,108.68
DLR/JUN22,128.40,128.98,128.99
DLR/MAY22,124.00,124.05,123.95
YPFD/FEB22,905.00,905.25,905.00
DLR/NOV22,152.50,153.00,NaN
DLR/OCT22,147.70,148.77,147.70
DLR/JUL22,133.30,133.70,130.00


,bid,offer
DLR/AGO22,0.471073,0.476981
DLR/ABR22,0.495558,0.496864
DLR/FEB22,0.409751,0.412714
DLR/JUN22,0.47944,0.489985
DLR/MAY22,0.492583,0.493751
YPFD/FEB22,0.218698,0.221663
DLR/NOV22,0.445236,0.449102
DLR/OCT22,0.451271,0.460714
DLR/JUL22,0.478176,0.484087
DLR/MAR22,0.491062,0.517559


,bid,offer
DLR/ABR22-0,0.495558,"(DLR/AGO22, 0.476981)"
DLR/ABR22-14,0.495558,"(DLR/JUN22, 0.489985)"
DLR/ABR22-17,0.495558,"(DLR/MAY22, 0.493751)"
DLR/ABR22-19,0.495558,"(YPFD/FEB22, 0.221663)"
DLR/ABR22-29,0.495558,"(DLR/NOV22, 0.449102)"
...,...,...
DLR/SEP22-13,0.460085,"(DLR/FEB22, 0.412714)"
DLR/SEP22-27,0.460085,"(YPFD/FEB22, 0.221663)"
DLR/SEP22-35,0.460085,"(DLR/NOV22, 0.449102)"
DLR/SEP22-61,0.460085,"(PAMP/FEB22, 0.153583)"
